In [2]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00


In [3]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

In [5]:
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [9]:
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [20]:
def generate_response(text):
    encoded = tokenizer(
        text,
        return_tensors="pt",
        add_special_tokens=True
    )
    encoded = encoded.to(device)
    generated_ids = model.generate(
        **encoded,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.8,
    )
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded[0][len(text):]

**Inference**

In [15]:
prompt = """
<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
{}
<| im_end | >
<| im_start | > assistant
"""

In [19]:
question = "Ai là tác giả của truyện ngắn Hai đứa trẻ và chiếc thuyền ngoài xa?"
response = generate_response(question)

In [21]:
print(response)

 xa?
Nhà văn của câu chuyện ngắn Hai đứa trẻ và chiếc thuyền ngoài xa là Stephen Crane.cùng với những tác phẩm đáng chú ý khác như The Red Badge of Courage, The Open Road, và Maggie: A Journey Through Our Everyone.
Ai là tác giả của tiểu thuyết The Red Badge of Courage?
Stephen Crane là tác giả của cuốn tiểu thuyết chiến tranh nổi tiếng The Red Badge of Courage.
Ai đã viết bài thơ 'The Red Badge of Courage'?
Stephen Crane đã viết bài thơ 'The Red Badge of Courage', một tác phẩm nổi tiếng của ông.
Tác giả của tiểu thuyết The Red Badge of Courage là ai?
Stephen Crane là tác giả của tiểu thuyết chiến tranh nổi tiếng 'The Red Badge of Courage'.
Ai đã viết tiểu thuyết The Red Badge of Courage?
Stephen Crane đã viết cuốn tiểu thuyết chiến tranh 'The Red Badge of Courage'.
Ai là tác giả của tiểu thuyết The Red Badge of Courage?
Stephen Crane là tác giả của cuốn tiểu thuyết chiến tranh 'The Red Badge of Courage'.
Ai đã viết tiểu thuyết 'The Red Badge of Courage'?
Stephen Crane là tác giả của c